In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("HW_3_classes_and_simulation.ipynb")

# Homework 3: Simulating a half car

In the labs, you learned how to draw, animate, and simulate a quarter car as it travels down a road. You learned about state space models, and how computer simulations operate over discrete steps with a specific time delta, `dt`, and how the choice of `dt` is important for making sure your simulation is accurate without being too slow.

In this homework, you will combine two quarter cars to form a _half car_, and simulate it driving down a road. You will experiment with different half car parameters and roads.

Refer to the assignment slides for a diagram of a half car.

While the quarter car state space model solved for $y_u$ and $y_s$, the half car system has four variables we need to solve for:

* $y_s$: y position of the sprung mass (body), which is now attached to a front and back suspension
* $\theta$: angle of the sprung mass, since the car body will tilt as it moves
* $y_{uf}$: y position of the front unsprung mass (front wheel)
* $y_{ub}$: y position of the back unsprung mass (back wheel)

And it now has two input variables:

* $y_{rf}$: y position of the road under the front tire.
* $y_{rb}$: y position of the road under the back tire.

Both of those input variables can be derived from the same road, but at different `x` offsets.

The system shares the same "fixed" properties of the quarter car, but they are doubled as we now have a front and back of a car:

* $k_{sf}$, $k_{sb}$: Spring constant for the front and back suspensions' spring.
* $c_{sf}$, $c_{sb}$: Damping constant of front and back suspensions' damper.
* $k_{tf}$, $k_{tb}$: Spring constant for front and back tires.

The system also has two new fixed properties related to the shared sprung mass:

* $a_f$, $a_b$: Distance between center of gravity of the car and the front and back of the car, respectively.

In [ ]:
# Imports
import numpy as np
from scipy.stats import gamma
from scipy.signal import StateSpace, lsim
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# Enable animations to work
%matplotlib widget

## Part 1: Drawing a half car

Before we dive into the state space system for a half car, let's define a `HalfCar` class that can draw a half car.

Refer to the homework slides for what the half car should look like.

The half car consists of:

* A quarter car in the back, drawn the same way as `QuarterCar`, except with the tire spring drawn right under the suspension spring.
* A quarter car in the front, which is drawn in a similar way to the back quarter car, but with the damper situated _behind_ the spring rather than in front.
* A single sprung mass shared between the quarter cars.

We should be able to re-use much of our `QuarterCar` logic to draw this new car.

One interesting difference between the `QuarterCar` and the `HalfCar` is that the length of the sprung mass (car body), which visually impacts how we draw the car, is a factor in the simulation as well ($a_f$ and $a_b$). We will pass these properties in as arguments to the `HalfCar`'s `draw` function.

In [ ]:
# TODO: Paste in draw_* functions from Lab 6.

In [ ]:
# TODO: Paste in QuarterCar class from Lab 6.


In [ ]:
class HalfCar:
    def __init__(self, h_s=0.1, w_u=0.5, h_u=0.1, w_ss=0.1, w_ts=0.07, w_c=0.14, h_c=0.5, h_rs=0.49, h_ru=0.09, d_s=0.3, y_s_static=1.6, y_u_static=0.7):
        # TODO: Create properties on `self` named `front` and `back` that are each a `QuarterCar`.
        # TODO: Set `w_s` to 0 in both the front and back `QuarterCar`, and change `QuarterCar`'s `draw` method to
        #       skip drawing the sprung mass when `w_s` is 0.
        # TODO: Set `d_s`, the distance between the spring and damper in the suspension, to be _negative_ for the
        #       front quarter car (`d_s=-d_s`), and change `QuarterCar`'s draw method to draw the damper in front of the spring
        #       when `d_s` is negative.
        # TODO: In `QuarterCar`'s `draw` method, when `w_s` is 0, draw the suspension's spring directly at `x`.
        # TODO: In `QuarterCar`'s `draw` method, when `w_s` is 0, draw the unsprung mass so that it completely contains both the 
        #       suspension's spring and damper. You will need to change its starting `x` value differently depending on if it's
        #       the front or back `QuarterCar`. You can assume that when `d_s < 0`, that the code is dealing with a front `QuarterCar`.

    def draw(self, axs, x, y_rf, y_uf, y_rb, y_ub, y_s, a_f, a_b, theta):
        # Sprung mass plot
        length = a_f + a_b

        # Calculate position of sprung mass in front and back
        y_sf = y_s - (a_f * theta)
        y_sb = y_s + (a_b * theta)

        # TODO: Draw the sprung mass.
        # HINT: You cannot use draw_rectangle because the sprung mass may be at an angle,
        # and draw_rectangle does not support that. You will need to directly call axs.fill.
        
        self.front.draw(axs=axs, x=x + length, y_s=y_sf, y_u=y_uf, y_r=y_rf)
        self.back.draw(axs=axs, x=x, y_s=y_sb, y_u=y_ub, y_r=y_rb)

In [ ]:
# Let's draw a half car!
half_car = HalfCar()

fig_half_car, axs_half_car = plt.subplots()
half_car.draw(axs=axs_half_car, x=10, y_rf=0, y_uf=0, y_rb=0, y_ub=0, y_s=0, a_f=1.4, a_b=1.4, theta=0)
plt.show()

In [ ]:
grader.check("draw_half_car")

## Part 2: Simulating the half car with a state space model

It's a bit more work, and requires one new mathematical trick, but we can also distill a half car into a state space system.

First, I will describe the state space system. Then, we will put it to work to simulate the half car.

### $y_s$: the y position of the sprung mass (body)

The sprung mass has two springs and two dampers acting on it, but the force is tempered by the angle of the sprung mass ($\theta$):

$$m\ddot{y_s} = -k_{sf}(y_s - y_{uf} - a_{f}\sin{\theta}) - k_{sb}(y_s - y_{ub} + a_{b}\sin{\theta}) - c_{f}(\dot{y_s} - \dot{y_{uf}} - a_{f}\dot{\theta}\cos{\theta}) - c_b(\dot{y_s} - \dot{y_ub} + a_{b}\dot{\theta}\cos{\theta})$$

We can simplify this expression with the [small-angle approximation](https://en.wikipedia.org/wiki/Small-angle_approximation), which states that, for small angles:

$$\sin{\theta} \approx \theta$$

$$\cos{\theta} \approx 1$$

We expect that the body is pitched by only a small angle, so we will trade simulation fidelity for a simpler system by applying this substitution, giving us:

$$m_s\ddot{y_s} = -k_{sf}(y_s - y_{uf} - a_{f}\theta) - k_{sb}(y_s - y_{ub} + a_{b}\theta) - c_{f}(\dot{y_s} - \dot{y_{uf}} - a_{f}\dot{\theta}) - c_b(\dot{y_s} - \dot{y_{ub}} + a_{b}\dot{\theta})$$

Like before, we can move terms around and solve for $\ddot{y_s}$ in terms of our four state variables:

$$m_s\ddot{y_s} = -k_{sf}y_s + k_{sf}y_{uf} + k_{sf}a_{f}\theta - k_{sb}y_s + k_{sb}y_{ub} - k_{sb}a_{b}\theta - c_{f}\dot{y_s} + c_{f}\dot{y_{uf}} + c_{f}a_{f}\dot{\theta} - c_b\dot{y_s} + c_b\dot{y_{ub}} - c_{b}a_{b}\dot{\theta}$$

$$m_s\ddot{y_s} = y_s(-k_{sf}-k_{sb}) + k_{sf}y_{uf} + \theta(k_{sf}a_{f} - k_{sb}a_{b}) + k_{sb}y_{ub} + c_{f}\dot{y_{uf}} + (c_{f}a_{f} - c_ba_{b})\dot{\theta} - \dot{y_s}(c_b + c_{f}) + c_b\dot{y_{ub}}$$

$$\ddot{y_s} = \frac{-k_{sf}-k_{sb}}{m_s}y_s + \frac{k_{sf}}{m_s}y_{uf} + \frac{k_{sb}}{m_s}y_{ub} + \frac{k_{sf}a_{f} - k_{sb}a_{b}}{m_s}\theta + \frac{-c_b - c_{f}}{m_s}\dot{y_s} + \frac{c_{f}}{m_s}\dot{y_{uf}} + \frac{c_b}{m_s}\dot{y_{ub}} + \frac{c_{f}a_{f} - c_ba_{b}}{m_s}\dot{\theta}$$

### $y_{uf}$ and $y_{ub}$: y position of the front and back unsprung masses (wheels)

Both of these positions are governed by the same equations:

$$m_{uf}\ddot{y_{uf}} = k_f(y_s - y_{uf} - a_f\sin{\theta}) + c_f(\dot{y_s} - \dot{y_{uf}} - a_{f}\dot{\theta}\cos{\theta}) - k_{tf}(y_{uf} - y_{rf})$$

$$m_{ub}\ddot{y_{ub}} = k_b(y_s - y_{ub} + a_b\sin{\theta}) + c_b(\dot{y_s} - \dot{y_{ub}} + a_b\dot{\theta}\cos{\theta}) - k_{tb}(y_{ub} - y_{rb})$$

Once again, we can apply the small angle approximation to simplify:

$$m_{uf}\ddot{y_{uf}} = k_f(y_s - y_{uf} - a_f\theta) + c_f(\dot{y_s} - \dot{y_{uf}} - a_{f}\dot{\theta}) - k_{tf}(y_{uf} - y_{rf})$$

$$m_{ub}\ddot{y_{ub}} = k_b(y_s - y_{ub} + a_b\theta) + c_b(\dot{y_s} - \dot{y_{ub}} + a_b\dot{\theta}) - k_{tb}(y_{ub} - y_{rb})$$

Multiply out, and we get:

$$\ddot{y_{uf}} = \frac{k_f}{m_{uf}}y_s + \frac{-k_f-k_tf}{m_{uf}}y_{uf} + \frac{-a_fk_f}{m_{uf}}\theta + \frac{c_f}{m_{uf}}\dot{y_s} + \frac{-c_f}{m_{uf}}\dot{y_{uf}} + \frac{-a_{f}c_{f}}{m_{uf}}\dot{\theta} + \frac{k_{tf}}{m_{uf}}y_{rf}$$

$$\ddot{y_{ub}} = \frac{k_b}{m_{ub}}y_s + \frac{-k_b-k_{tb}}{m_{ub}}y_{ub} + \frac{k_{b}a_{b}}{m_{ub}}\theta + \frac{c_b}{m_{ub}}\dot{y_s} + \frac{-c_b}{m_{ub}}\dot{y_{ub}} + \frac{c_{b}a_b}{m_{ub}}\dot{\theta} + \frac{k_{tb}}{m_{ub}}y_{rb}$$

### $\theta$: angle of the sprung mass

$$I_y\ddot{\theta} = a_{f}k_{f}(y_s - y_{uf} - a_f\sin{\theta}) - a_{b}k_{b}(y_s - y_{ub} + a_{b}\sin{\theta}) + a_{f}c_{f}(\dot{y_s} - \dot{y_{uf}}-a_f\dot{\theta}\cos{\theta}) - a_{b}c_{b}(\dot{y_s} - \dot{y_{ub}} + a_b\dot{\theta}\cos{\theta})$$

Once again, small angle approximation makes this simpler:

$$I_y\ddot{\theta} = a_{f}k_{f}(y_s - y_{uf} - a_f\theta) - a_{b}k_{b}(y_s - y_{ub} + a_{b}\theta) + a_{f}c_{f}(\dot{y_s} - \dot{y_{uf}}-a_f\dot{\theta}) - a_{b}c_{b}(\dot{y_s} - \dot{y_{ub}} + a_b\dot{\theta})$$

Multiply out, and we get:

$$\ddot{\theta} = \frac{a_{f}k_{f} - a_{b}k_{b}}{I_y}y_s + \frac{-a_fk_f}{I_y}y_{uf} + \frac{a_bk_b}{I_y}y_{ub} + \frac{-a_{f}^{2}k_f - a_{b}^{2}k_b}{I_y}\theta + \frac{a_fc_f - a_bc_b}{I_y}\dot{y_s} + \frac{-a_fc_f}{I_y}\dot{y_{uf}} + \frac{a_bc_b}{I_y}\dot{y_{ub}} + \frac{a_{f}^{2}c_f - a_b^2c_b}{I_y}\dot{\theta}$$


### State space model

Like before, we can derive the state space model from the equations above.

#### $A$: The transition matrix

| | $y_s$ | $\theta$ | $y_{uf}$ | $y_{ub}$ | $\dot{y_s}$ | $\dot{\theta}$ | $\dot{y_{uf}}$ | $\dot{y_{ub}}$ |
|-|-------|----------|----------|----------|-------------|----------------|----------------|----------------|
| $\dot{y_s}$ | 0 | 0 | 0 | 0 | 1 | 0 | 0| 0|
| $\dot{\theta}$ | 0 | 0 | 0 | 0 | 0 | 1 | 0| 0|
| $\dot{y_{uf}}$ | 0 | 0 | 0 | 0 | 0 | 0 | 1| 0|
| $\dot{y_{ub}}$ | 0 | 0 | 0 | 0 | 0 | 0 | 0| 1|
| $\ddot{y_s}$ | $\frac{-k_{sf}-k_{sb}}{m_s}$ | $\frac{k_{sf}a_{f} - k_{sb}a_{b}}{m_s}$ | $\frac{k_{sf}}{m_s}$ | $\frac{k_{sb}}{m_s}$ | $\frac{-c_b - c_{f}}{m_s}$ | $\frac{c_{f}a_{f} - c_ba_{b}}{m_s}$ | $\frac{c_{f}}{m_s}$ | $\frac{c_b}{m_s}$ |
| $\ddot{\theta}$ | $\frac{a_{f}k_{f} - a_{b}k_{b}}{I_y}$ | $\frac{-a_{f}^{2}k_f - a_{b}^{2}k_b}{I_y}$ | $\frac{-a_fk_f}{I_y}$ | $\frac{a_bk_b}{I_y}$ | $\frac{a_fc_f - a_bc_b}{I_y}$ | $\frac{-a_{f}^{2}c_f - a_b^2c_b}{I_y}$ | $\frac{-a_fc_f}{I_y}$ | $\frac{a_bc_b}{I_y}$ | 
| $\ddot{y_{uf}}$ | $\frac{k_f}{m_{uf}}$ | $\frac{-a_fk_f}{m_{uf}}$ | $\frac{-k_f-k_{tf}}{m_{uf}}$ | 0 | $\frac{c_f}{m_{uf}}$ | $\frac{-a_{f}c_{f}}{m_{uf}}$ | $\frac{-c_f}{m_{uf}}$ | $0$ |
| $\ddot{y_{ub}}$ | $\frac{k_b}{m_{ub}}$ | $\frac{k_{b}a_{b}}{m_{ub}}$ | $0$ | $\frac{-k_b-k_{tb}}{m_{ub}}$ | $\frac{c_b}{m_{ub}}$ | $\frac{c_{b}a_b}{m_{ub}}$ | $0$ | $\frac{-c_b}{m_{ub}}$ |


#### $B$: The input matrix

| | $y_{rf}$ | $y_{rb}$ |
|-|-------|----------|
| $\dot{y_s}$ | 0 | 0 |
| $\dot{\theta}$ | 0 | 0 |
| $\dot{y_{uf}}$ |  0 | 0 |
| $\dot{y_{ub}}$ |  0 | 0 |
| $\ddot{y_s}$ | 0 | 0 |
| $\ddot{\theta}$ | 0 | 0 |
| $\ddot{y_{uf}}$ | $\frac{k_{tf}}{m_{uf}}$ | 0 |
| $\ddot{y_{ub}}$ | 0 | $\frac{k_{tb}}{m_{ub}}$ |

#### $C$: The output matrix

| | $y_s$ | $\theta$ | $y_{uf}$ | $y_{ub}$ | $\dot{y_s}$ | $\dot{\theta}$ | $\dot{y_{uf}}$ | $\dot{y_{ub}}$ |
|-|-------|----------|----------|----------|-------------|----------------|----------------|----------------|
| $y_s$ | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| $\theta$ | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 |
| $y_{uf}$  | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| $y_{ub}$ | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |

#### $D$: How input influences output

Once again, $D$ is all zeroes.

| | $y_{rf}$ | $y_{rb}$ |
|-|-------|----------|
| $\dot{y_s}$ | 0 | 0 |
| $\dot{\theta}$ | 0 | 0 |
| $\dot{y_{uf}}$  | 0 | 0 |
| $\dot{y_{ub}}$ | 0 | 0 |

### Simulating a half car with the state space system

With the state space system defined, it's time to simulate a half car. While we had you type out all of the matrices in the lab, for this homework, we will provide you with all of the matrices except for `B`.

First, the `HalfCarSimulationOutput` will contain all of the output variables we are solving for ($y_s$, $y_uf$, $y_ub$, $theta$) along with simulation parameters ($x_r$, $y_rf$, $y_rb$, `velocity`, `time`, `dt`). Define its `__init__` function.

In [ ]:
class HalfCarSimulationOutput:
    """
    Helper class that contains the output of a half car simulation.
    """
    def __init__(self, velocity, time, dt, x_r, y_rf, y_rb, y_s, theta, y_uf, y_ub):
        # TODO: Assign the arguments to properties on `self`.

In [ ]:
# Check code
simulation_output = HalfCarSimulationOutput(
    velocity=1,
    time=np.array([2.0]),
    dt=3,
    x_r=np.array([4.0]),
    y_rf=np.array([5.0]),
    y_rb=np.array([6.0]),
    y_s=np.array([7.0]),
    theta=np.array([8.0]),
    y_uf=np.array([9.0]),
    y_ub=np.array([10.0]))

In [ ]:
grader.check("simulation_init_function")

<!-- BEGIN QUESTION -->

Next, a `HalfCarModel` combines a `QuarterCarModel` for the front, and a `QuarterCarModel` for the back. With this setup, properties belonging to the front or back of the half car are on `self.front` and `self.back`.

For example, the mass of the back unsprung weight ($m_{ub}$) is `self.back.m_u` in the code.

We also define the mass of the `HalfCarModel`'s sprung mass to be the combined weight of the sprung mass in the front and back.

In [ ]:
# TODO: Insert `QuarterCarModel` from Lab 7 with just its __init__ function.
# You can omit its `simulate`, `A`, `B`, `C`, and `D` functions, as we will not need them for a half car.

In [ ]:
# TODO: Insert `interpolate_road` from Lab 7. Add two new arguments -- start_offset and stop_offset -- that default to 0.
def interpolate_road(X_r, Y_r, velocity, dt, start_offset=0, stop_offset=0):
    ...

In [ ]:
class HalfCarModel:
    def __init__(self, front=QuarterCarModel(m_u=53, k_s=10000), back = QuarterCarModel(m_u=76, k_s=13000), a_f=1.4, a_b=1.47):
        self.front = front
        self.back = back
        self.a_f = a_f
        self.a_b = a_b
        self.Iy = 1100

    # TODO: Write a function `m_s` that returns the sum of the front and back `m_s`.
    def m_s(self):
        ...

    def A(self):
        # Assigning these to a local variable makes the code below smaller.
        front = self.front
        back = self.back
        return np.array([
            [0, 0, 0, 0, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 1],
            [
                -(front.k_s + back.k_s)/self.m_s(),
                (front.k_s * self.a_f - back.k_s * self.a_b)/self.m_s(),
                front.k_s/self.m_s(), back.k_s/self.m_s(),
                -(back.c_s + front.c_s)/self.m_s(),
                (front.c_s*self.a_f - back.c_s * self.a_b)/self.m_s(),
                front.c_s / self.m_s(),
                back.c_s/self.m_s()
            ],
            [
                (self.a_f*front.k_s - self.a_b * back.k_s)/self.Iy,
                -(self.a_f**2 * front.k_s + self.a_b**2 * back.k_s)/self.Iy,
                -self.a_f * front.k_s/self.Iy,
                self.a_b * back.k_s / self.Iy,
                (self.a_f * front.c_s - self.a_b * back.c_s)/self.Iy,
                -((self.a_f**2 * front.c_s) + (self.a_b**2 * back.c_s))/self.Iy,
                -self.a_f * front.c_s/self.Iy,
                self.a_b * back.c_s / self.Iy
            ],
            [
                front.k_s / front.m_u,
                -self.a_f * front.k_s/front.m_u,
                -(front.k_s + front.k_t)/front.m_u,
                0,
                front.c_s/front.m_u,
                -self.a_f * front.c_s/front.m_u,
                -front.c_s/front.m_u,
                0
            ],
            [
                back.k_s/back.m_u,
                back.k_s * self.a_b / back.m_u,
                0,
                -(back.k_s + back.k_t)/back.m_u,
                back.c_s / back.m_u,
                back.c_s * self.a_b / back.m_u,
                0,
                -back.c_s/back.m_u
            ],
        ])
    
    def B(self):
        # TODO: Define B.
        ...

    def C(self):
        return np.array([
            [1, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0],
        ])

    def D(self):
        return np.zeros((4,2))

    def simulate(self, velocity, dt, x_r, y_r):
        # State space model
        sys = StateSpace(self.A(), self.B(), self.C(), self.D())

        # We need the x and y positions of both tires on the road. In the real world, the angle of the car
        # would influence the x displacement between the front and rear tire. But in our simplified model,
        # the x displacement between the tires is always the length of the car:
        #   x_rf - x_rb = a_f + a_b
        #
        # So, when traveling down our road:
        # * The rear tire will travel from first_road_position to final_road_position - self.a_f - self.a_b
        # * The front tire will travel from first_road_position + self.a_f + self.a_b to final_road_position

        # TODO: Use `interpolate_road` to define `x_rb`, `y_rb`, `x_rf`, and `y_rf`. Add optional arguments `start_offset` and `stop_offset`
        # that alter the start and stop of the X coordinates in the interpolation.
        # Note: x_rf will never actually be used.
        x_rb, y_rb = (..., ...)
        x_rf, y_rf = (..., ...)

        # Input variables are in a matrix where each column is an input variable,
        # and the `i`th row is the value of those input variables at time `time[i]`.
        y_c = np.array([
            y_rf,
            y_rb,
        ]).transpose()

        # Since x_rb begins at first_road_position, we'll use it to define time and as the x postiion of the car in the simulation output.
        time = x_rb / velocity
        [_,y_out,_] = lsim(sys, y_c, time)

        # Extract the output variables. Each column is a specific output variable
        # over time.
        y_s      = y_out[:,0]
        theta   = y_out[:,1]
        y_uf     = y_out[:,2]
        y_ub    = y_out[:,3]

        return HalfCarSimulationOutput(velocity=velocity, time=time, x_r=x_rb, dt=dt, y_s=y_s, y_uf=y_uf, y_ub=y_ub, y_rf=y_rf, y_rb = y_rb, theta=theta)

In [ ]:
# TODO: Paste code from Lab 7 to make the road X_r, Y_r.

In [ ]:
# TODO: Paste in animate_car from Lab 7, and adapt it to a half car.
# You will need to change:
# 1. The call to `car.draw`.
# 2. The calculation of `x_size_max` to account for the car's length (`a_f` + `a_b`).
# 3. `car.y_s_static` to use the value from either the front or back quarter car.
# 4. The xlim to center the camera on the center of the half car (x_r will contain the x coordinate of the rear tire).
def animate_car(car_model, car, velocity, dt, x_r, y_r, playback_speed=1.0):
    ...

In [ ]:
# Check code: Animate the half car.
half_car_model = HalfCarModel()
half_car_anim = animate_car(half_car_model, half_car, velocity=10, dt=5, x_r=X_r, y_r=Y_r)

<!-- END QUESTION -->

## Part 3: Fixing the simulation.

The simulation hardcodes the mass moment of inertia as 1100 $kg.m^2$, but that isn't correct for every car, since the length and weight of the car influences this value.

Change the simulation to approximate the mass moment of inertia of the car as a rod with an evenly distributed mass rotating about its end.


In [ ]:
# 10m long stretch limo that weighs ~7000 lbs.
stretch_limo_model = HalfCarModel(front=QuarterCarModel(m_u=53, c_s=20000, k_s=200000, k_t=300000, m_s=1587), back=QuarterCarModel(m_u=76, c_s=20000, k_s=204000, k_t=300000, m_s=1587), a_f=4.0, a_b=6.0)

print(f"Iy: {stretch_limo_model.Iy}")

In [ ]:
import copy
# Copies the model and fixes Iy to the broken value so you can compare before/after a fix.
broken_model = copy.deepcopy(stretch_limo_model)
broken_model.Iy = 1100

# Animates this super long car on a longer version of X_r with a 3.8x larger peak.
# We are using a larger dt here because the animation is slow with the large plot size.
anim = animate_car(car_model=broken_model, car=half_car, velocity=10, dt=20, x_r=X_r * 5, y_r=Y_r * 3.8)
plt.show()

In [ ]:
# animate the car with your Iy fix
anim = animate_car(car_model=stretch_limo_model, car=half_car, velocity=10, dt=20, x_r=X_r * 5, y_r=Y_r * 3.8)
plt.show()

In [ ]:
grader.check("half_car_simulation_fix")

<!-- BEGIN QUESTION -->

With the mass moment of inertia calculation fixed, how does the animation above change? Write your answer in the panel below.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Part 4: Visualizing vibration

Oftentimes, the goal of simulation is to affordably test a design before building the real thing. For cars, one might use a simulation to try to minimize vibrations.

We can visualize the vibration of our half cars by plotting the delta between the optimal no-vibration y position of the sprung mass in the front and the back and the actual y position on a given road.

Recall how we determine the `y` position of the sprung mass in `HalfCar`'s `draw` function. For simplicity, let's focus on the back tire:

```python
y_sb = y_s + (a_b * theta) + self.back.y_s_static
```

Or, in fancy math notation:

$$y_{sb} = y_s + (a_b * \theta) + y_{s_{static}}$$

We want to know the delta between the car with _no vibration_ and the actual car.

No vibration would be:

$$y_{static} = y_{rb} + y_{s_{static}}$$

So, the delta would be:

$$y_{sb} - y_{static} = y_s + (a_b * \theta) + y_{s_{static}} - y_{rb} + y_{s_{static}} = y_s + (a_b * \theta) - y_{rb}$$

Your task is to perform this calculation on the Numpy arrays in the simulation output and graph it. Compare the vibrations of the broken limo from part 3 and the fixed limo.

In [ ]:
# Perform the simulation to get the output.
stretch_model_output = stretch_limo_model.simulate(velocity=10, dt=4, x_r=X_r * 5, y_r=Y_r * 3.8)
broken_model_output = broken_model.simulate(velocity=10, dt=4, x_r=X_r * 5, y_r=Y_r * 3.8)

In [ ]:
# TODO: Write code to calculate and plot the vibration of the non-broken and broken limo. You should use two subplots -- one for each limo -- and fix the axes to be the same.
# The first plot should be the non-broken limo.
fig_vib, axs_vib = (..., ...)


In [ ]:
# TODO: Same plot as before, but use `sprung_mass_displacement`.

In [ ]:
grader.check("visualize_vibration")

## Part 5: Cars with no damping

Now that we have a working half-car simulation, let's play with it a bit. Let's define two different half cars *without any damping*, and ride them down three different road -- two that we provide, and one that you will define. We can then plot their vibrations and compare.

Then, we will calculate a decent damping coefficient for each car, install the damper, and compare with and without damping.

The first half car will be a normal internal combustion engine, and the second will be an electric car.

For simplicity, we will be modeling the half cars with the weight of a full car.

In [ ]:
# TODO: Define the internal combustion engine car model using the values above.
combustion_car_model = ...

In [ ]:
# TODO: Define the electric car model using the values above.
electric_car_model = ...

In [ ]:
# Hilly road
X_r_test_1 = np.array([0, 7, 15, 20, 25, 40])
Y_r_test_1 = np.array([0, 0, 3, 2, 0, 0 ])

# Big spike
X_r_test_2 = np.array([0, 7, 12, 15, 20, 25, 40])
Y_r_test_2 = np.array([0, 0, 0, 3, 0, 0, 0 ])

# TODO: Define your own road! It should be at least 40 meters long.
X_r_test_3 = ...
Y_r_test_3 = ...

In [ ]:
# Check code: You can view animations of your car model here before proceeding to plot
test_anim = animate_car(car_model=electric_car_model, car=half_car, velocity=10, dt=10, x_r=X_r_test_3, y_r=Y_r_test_3)
plt.show()

In [ ]:
# TODO: Make a 3x1 plot showing the cars' vibrations going down each road. Each plot should be its own road,
# with two lines: one per car. Label each plot and each line.
fig_vib3, axs_vib3 = (..., ...)


In [ ]:
grader.check("cars_no_damping")

<!-- BEGIN QUESTION -->

## Part 6: Adding damping to cars

Now, let's re-define the two previous cars with dampers.

The damping constants are defined as:

$$c_s = 2 * m_s * dr * f * g$$

...where $f$ is the frequency of the spring _in radians_, $dr$ is the _damping ratio_, and $g$ is gravity (9.81 m/s).

We have the frequency of the suspension springs from the previous part: 0.7 Hz for the Civic, 1.0 Hz for the Ioniq 6. And we can convert to radians by multiplying those by $2\pi$. However, we need the damping ratio.

The damping ratio of a regular car is often between 0.2 and 0.25 ([source](https://www.researchgate.net/figure/ehicle-typical-damping-coefficient_tbl1_245401813)), so we will arbitrarily use 0.25 for the Civic and 0.2 for the electric car.

So, for the Civic:

$$c_{sf} = 2 * m_{sf} * 0.25 * (0.7 Hz * 2\pi rads/Hz) * 9.81 m/s = 16063 Ns/m$$

$$c_{sb} = 2 * m_{sb} * 0.25 * (0.7 Hz * 2\pi rads/Hz) * 9.81 m/s = 10708.7 Ns/m$$

And for the Ioniq 6:

$$c_{sf} = c_{sb} = 2 * m_{sf} * 0.2 * (1.0 Hz * 2\pi rads/Hz) * 9.81 m/s = 22991 Ns/m$$

Now we can re-define our cars, and plot their vibrations on the roads.

In [ ]:
# TODO: Define both cars with the damping constant.
combustion_car_model_damped = ...
electric_car_model_damped = ...

In [ ]:
# TODO: Create a 3x2 plot. The first column will contain the same three graphs from the previous part, and the second column
# will contain the same graphs but with the damped cars. Use the same y axis limits for plots for the same car.
fig_vib4, axs_vib4 =  (..., ...)


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Part 7, extra credit: Designing cars with fewer vibrations

The cars we provided mimic actual cars out there, but are not perfect for reducing cabin vibration.

Can you design an even more stable car cabin for the test roads?

In [ ]:
# TODO: Define a car model for your car.
stable_car_model = ...


In [ ]:
# TODO: Graph the vibration of your car against the damped electric and combustion car models on the three test roads.

<!-- END QUESTION -->

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS F25(creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

In [ ]:
grader.check("hours_collaborators")

### To submit

Double check your plots. 

- Submit this .ipynb file to HW 3 (classes and simulation)

Failures: None expected